# Mapping Longitude and Latitude to GeoJSON polygon

## Reference:
https://stackoverflow.com/questions/20776205/point-in-polygon-with-geojson-in-python

In [9]:
import json
from shapely.geometry import shape, Point
mappath = '/home/lmtdreamer/NYCTaxi/nyu-2451-36743-geojson.json'
readpath = '/media/sf_Temp/Disk_E-NYC/5-3rd_cleaned/Green/cleaned_2013_Green.csv'
writepath = '/media/sf_Temp/Disk_E-NYC/6-convertLonLat/2013_Green_map.csv'

In [11]:
def cal_zone(mappath,readpath,writepath,windows=True):
    axis = ('PU_Lon','PU_Lat','DO_Lon','DO_Lat')
    rf = open(readpath,'r') # 開啟檔案
    head = rf.readline() # 讀取一行 (row) 之後換行
    head = head.split(',') # 每行的欄位用 , 分隔
    if windows:
        head[-1] = head[-1][:-1] #去除末端的換行符號
    wf = open(writepath, "w")
    wf.write('row_id,PULocationID,DOLocationID\n') # NOT withcolumn (cause not PySpark)?  Also, only 2 columns without other data from original CSV?
    for i in range(len(head)):
        if head[i] == axis[0]:
            pu_lon_ = i
        elif head[i] == axis[1]:
            pu_lat_ = i
        elif head[i] == axis[2]:
            do_lon_ = i
        elif head[i] == axis[3]:
            do_lat_ = i
    with open(mappath) as f:
                        js = json.load(f)
    try:
        row_cnt=0
        row_id = 0
        while True:
            line = rf.readline() # head = rf.readline()???
            if line:
                row_id += 1
                row = line.split(',') # head = head.split(',')???
                if windows:
                    row[-1] = row[-1][:-1] # 去除末端的換行符號
                if len(row)==len(head):
                    new_row=''
                    pu_lon = row[pu_lon_]
                    pu_lat = row[pu_lat_]
                    do_lon = row[do_lon_]
                    do_lat = row[do_lat_]
                    point1 = Point(float(pu_lon), float(pu_lat))
                    point2 = Point(float(do_lon), float(do_lat))
                    for feature in js['features']:
                        polygon = shape(feature['geometry'])
                        if polygon.contains(point1):
                            zone1 = str(feature['properties']['locationid']) # zone1 or PULocationID???
                        if polygon.contains(point2):
                            zone2 = str(feature['properties']['locationid']) # zone2 or DOLocationID???
                            new_row = str(row_id)+','+zone1+','+zone2 # PULocationID+','+DOLocationID???
                    wf.write(new_row+'\n')
                    row_cnt+=1
            else:
                print('rows:',str(row_cnt))
                break
    finally:
        rf.close()
        wf.close()            

In [3]:
# 檢查讀取到的檔案格式

axis = ('PU_Lon','PU_Lat','DO_Lon','DO_Lat')
rf = open(readpath,'r')
head = rf.readline().split(',')
print(head)
line = rf.readline().split(',')
line[-1] = line[-1][:-1]
for i in line:
    print(i)
rf.close()

['PU_Time', 'DO_Time', 'Rate_Code', 'DO_Lat', 'Trip_Distance', 'Fare_Amt', 'Extra', 'Tip_Amt', 'PU_Lon', 'PU_Lat', 'DO_Lon\n']
12/06/2013 12:11:05 PM
12/06/2013 12:14:06 PM
1
40.811981201171875
0.5
4
0
0
-73.965065002441406
40.806140899658203
-73.962234497070312


In [4]:
# 測試單行讀取載客上車經緯度與下車經緯度

rf = open(readpath,'r')
head = rf.readline().split(',')
line = rf.readline().split(',')
line[-1] = line[-1][:-1]
x,y=line[-1],line[3]
x1,y1=line[-3],line[-2]
point1 = Point(float(x), float(y))
point2 = Point(float(x1), float(y1))
with open(mappath) as f:
    js = json.load(f)
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point1):
            zone1 = str(feature['properties']['locationid']) # zone1 or PULocationID???
        if polygon.contains(point2):
            zone2 = str(feature['properties']['locationid']) # zone2 or DOLocationID???
    new_row = zone1+','+zone2 # PULocationID+','+DOLocationID???
    print(new_row)

166,166


In [15]:
cal_zone(mappath,readpath,writepath)

rows: 1210811


In [1]:
! ls /media/sf_Temp/Disk_E-NYC/6-convertLonLat/

2013_Green_map.csv  2013_Green_map-test-Local.csv


In [ ]:
! cat /media/sf_Temp/Disk_E-NYC/6-convertLonLat/2013_Green_map.csv

In [7]:
p = '/media/sf_Temp/Disk_E-NYC/5-3rd_cleaned/Green/cleaned_2013_Green.csv'
rf = open(p,'r') # 開啟檔案
for i in range(117804):
    head = rf.readline() # 讀取一行 (row) 之後換行
head = head.split(',')
head[-1] = head[-1][:-1]
print(head)
for i in range(3):
    head = rf.readline()
    head = head.split(',')
    head[-1] = head[-1][:-1]
    print(head)

['11/30/2013 07:41:33 PM', '11/30/2013 08:00:43 PM', '1', '40.769596099853516', '4.91', '19', '0', '3.8', '-73.951805114746094', '40.810401916503906', '-73.98931884765625']
['12/11/2013 02:25:03 PM', '12/11/2013 02:35:46 PM', '1', '40.682834625244141', '0.88', '8', '0', '0', '-73.982772827148438', '40.688484191894531', '-73.970779418945313']
['12/15/2013 06:41:25 AM', '12/15/2013 06:45:51 AM', '1', '40.917964935302734', '1.04', '5.5', '0', '0', '-73.902175903320312', '40.931411743164063', '-73.901054382324219']
['11/28/2013 06:28:03 PM', '11/28/2013 06:38:27 PM', '1', '40.801395416259766', '2.04', '9.5', '0', '1.5', '-73.93890380859375', '40.805084228515625', '-73.965950012207031']


In [19]:
point = Point(-73.93890380859375, 40.805084228515625)
for feature in js['features']:
    polygon = shape(feature['geometry'])
    if polygon.contains(point):
        zone = feature['properties']['locationid']
        print(zone)
        

74


In [ ]:
# mappath = 'nyu-2451-36743-geojson.json'

### Original code to map (Longitude,Latitude) set to NYC Taxi Zones

In [10]:
# load GeoJSON file containing sectors
with open('nyu-2451-36743-geojson.json') as f:
    js = json.load(f)

In [ ]:
len(js['features'])

In [ ]:
# 讀取全部 263 區域
# js['features']

In [ ]:
js['features'][1]

In [ ]:
# construct point based on lon/lat returned by geocoder
# 隨便給一個點的經緯度
point = Point(-73.9919569999999, 40.721567)

In [ ]:
type(point)

In [ ]:
# check each polygon to see if it contains the point
# 看看剛剛給的點是不是在某一個區域中

for feature in js['features']:
    polygon = shape(feature['geometry'])
    if polygon.contains(point):
        zone = feature['properties']['locationid']